In [5]:
import numpy as np
from scipy.sparse import random, csr_matrix
import Scripts.simulate_p_circuit as simulate_p_circuit
import Scripts.CustomScripts as CS
import os
J, h, node_order = CS.load_npz_data("8bIntFac_v6.npz", target_folder="Circuit_Library")

# Convert J to Compressed Sparse Row (CSR) format
J_sparse = csr_matrix(J)

# Extract CSR components
values = J_sparse.data
col_indices = J_sparse.indices
row_ptr = J_sparse.indptr
print(J.size)
print(values)
print(len(values))
print(col_indices)
print(row_ptr)

# # Function to write .mem file in hexadecimal format
# def write_mem_file(filename, data_array):
#     with open(filename, "w") as f:
#         for value in data_array:
#             f.write(f"{int(value) & 0xFF:02X}\n")  # Convert to 8-bit hex

# # Save files
# write_mem_file("values.mem", values)
# write_mem_file("col_indices.mem", col_indices)
# write_mem_file("row_ptr.mem", row_ptr)
# write_mem_file("h.mem", h)

# print("Sparse matrix data successfully saved as .mem files for Vivado!")


Found folder: C:\repositories\ProbabilisticComputing\Circuit_Library
8281
[-1.  2. -2. -2. -1.  2. -2.  3. -2. -2.  2. -2.  3.  2.  2. -1.  2. -1. -2.  2. -2.  2. -2.  2.  2. -1. -2. -3. -2.  3.  2.  2. -2.  3.  2.  2.  3.  3. -1.  2. -2.  3. -3.  3. -2. -1.  2. -2. -2.
  3. -1.  3.  2. -1.  2. -3.  2.  2. -2.  3. -2. -1. -2. -1.  2.  2.  3.  3. -1.  2. -2.  2.  2.  2. -2.  2.  2. -1. -3. -2.  3. -2. -2. -2. -1. -2.  2.  2. -3. -2.  3. -3.  3. -2. -2. -2. -3.  3.
 -2. -2. -2.  3.  3. -1. -2. -1.  2. -2. -2. -1. -2. -2. -2. -3. -2.  3. -3. -2.  3. -2. -2. -2. -2. -1. -2. -1.  3.  3. -1. -2.  2. -3. -2. -2. -2.  3. -2. -2. -1.  2.  3.  3. -1.  2. -1. -2.  2.
  2. -3. -2.  3. -3. -2.  3.  2.  2. -1. -2. -2. -1.  2. -2.  3.  3. -1.  2. -3.  2.  2. -2.  3. -2. -2. -3.  3. -2. -1.  2. -2. -2. -3. -2.  3. -3. -2.  3.  2.  2.  2.  2. -3. -2.  3. -3.  3. -2.
 -2. -2. -1.  2.  2.  2.  3.  3. -1. -2. -1. -2. -2. -2. -1. -2. -3. -2. -2. -2.  3. -2. -2. -1.  2.  3.  3. -1.  2. -1. -2. -3.  2.  2. -

In [6]:
# Function to perform Run-Length Encoding (RLE)
def run_length_encode(arr):
    values_compressed = []
    run_lengths = []
    
    prev_value = arr[0]
    count = 1
    for i in range(1, len(arr)):
        if arr[i] == prev_value:
            count += 1
        else:
            values_compressed.append(prev_value)
            run_lengths.append(count)
            prev_value = arr[i]
            count = 1
    values_compressed.append(prev_value)
    run_lengths.append(count)
    
    return np.array(values_compressed), np.array(run_lengths)

# Apply RLE encoding
encoded_values = run_length_encode(values)

# Print in the desired format
print(encoded_values)

(array([-1.,  2., -2., -1.,  2., -2.,  3., -2.,  2., -2.,  3.,  2., -1.,  2., -1., -2.,  2., -2.,  2., -2.,  2., -1., -2., -3., -2.,  3.,  2., -2.,  3.,  2.,  3., -1.,  2., -2.,  3., -3.,  3., -2.,
       -1.,  2., -2.,  3., -1.,  3.,  2., -1.,  2., -3.,  2., -2.,  3., -2., -1., -2., -1.,  2.,  3., -1.,  2., -2.,  2., -2.,  2., -1., -3., -2.,  3., -2., -1., -2.,  2., -3., -2.,  3., -3.,  3.,
       -2., -3.,  3., -2.,  3., -1., -2., -1.,  2., -2., -1., -2., -3., -2.,  3., -3., -2.,  3., -2., -1., -2., -1.,  3., -1., -2.,  2., -3., -2.,  3., -2., -1.,  2.,  3., -1.,  2., -1., -2.,  2.,
       -3., -2.,  3., -3., -2.,  3.,  2., -1., -2., -1.,  2., -2.,  3., -1.,  2., -3.,  2., -2.,  3., -2., -3.,  3., -2., -1.,  2., -2., -3., -2.,  3., -3., -2.,  3.,  2., -3., -2.,  3., -3.,  3.,
       -2., -1.,  2.,  3., -1., -2., -1., -2., -1., -2., -3., -2.,  3., -2., -1.,  2.,  3., -1.,  2., -1., -2., -3.,  2., -2.,  3., -2., -1., -2., -1.,  2., -2.,  3., -1.,  2., -3.,  2., -2.,  3.,
       -2., -1

In [7]:
# Function to generate `.mem` files for RLE-encoded sparse matrix
def generate_rle_mem_files(J_bipolar, h_bipolar, file_prefix):
    # Ensure the "bram" folder exists
    bram_folder = "bram"
    if not os.path.exists(bram_folder):
        os.makedirs(bram_folder)

    # Convert bipolar J and h to binary fixed-point values
    J_binary, h_binary = J_bipolar, h_bipolar  # Assuming already in integer form

    # Convert J to Compressed Sparse Row (CSR) format
    from scipy.sparse import csr_matrix
    J_sparse = csr_matrix(J_binary)

    # Extract CSR components
    values = J_sparse.data
    col_indices = J_sparse.indices
    row_ptr = J_sparse.indptr
    print(values)
    print(col_indices)
    print(row_ptr)
    # Apply Run-Length Encoding (RLE) to values[]
    values_compressed, run_lengths = run_length_encode(values)
    print(run_lengths)
    # Construct file paths
    values_file = os.path.join(bram_folder, f"{file_prefix}_values.mem")
    run_lengths_file = os.path.join(bram_folder, f"{file_prefix}_run_lengths.mem")
    col_indices_file = os.path.join(bram_folder, f"{file_prefix}_col_indices.mem")
    row_ptr_file = os.path.join(bram_folder, f"{file_prefix}_row_ptr.mem")
    h_file = os.path.join(bram_folder, f"{file_prefix}_h.mem")

    # Write the values memory file (RLE compressed)
    with open(values_file, "w") as f:
        for v in values_compressed:
            bin_value, _ = CS.decimal_to_s4_3(v)
            f.write(bin_value + "\n")

    # Write the run_lengths memory file
    with open(run_lengths_file, "w") as f:
        for r in run_lengths:
            f.write(f"{r:08b}\n")  # Store run lengths as 8-bit binary

    # Write the col_indices memory file
    with open(col_indices_file, "w") as f:
        for c in col_indices:
            f.write(f"{c:08b}\n")  # Store column indices as 8-bit binary

    # Write the row_ptr memory file (Delta Encoded)
    with open(row_ptr_file, "w") as f:
        for r in row_ptr:
            f.write(f"{r:016b}\n")  # Store as 16-bit binary

    # Write the h memory file (Bias vector)
    with open(h_file, "w") as f:
        for i in range(h_binary.shape[0]):
            bin_value, _ = CS.decimal_to_s4_3(h_binary[i])
            f.write(bin_value + "\n")

    print(f"RLE-encoded mem files generated in folder '{bram_folder}':")
    print(f"  {file_prefix}_values.mem")
    print(f"  {file_prefix}_run_lengths.mem")
    print(f"  {file_prefix}_col_indices.mem")
    print(f"  {file_prefix}_row_ptr.mem")
    print(f"  {file_prefix}_h.mem")

In [8]:
generate_rle_mem_files(J,h,"4bIntFac")

[-1.  2. -2. -2. -1.  2. -2.  3. -2. -2.  2. -2.  3.  2.  2. -1.  2. -1. -2.  2. -2.  2. -2.  2.  2. -1. -2. -3. -2.  3.  2.  2. -2.  3.  2.  2.  3.  3. -1.  2. -2.  3. -3.  3. -2. -1.  2. -2. -2.
  3. -1.  3.  2. -1.  2. -3.  2.  2. -2.  3. -2. -1. -2. -1.  2.  2.  3.  3. -1.  2. -2.  2.  2.  2. -2.  2.  2. -1. -3. -2.  3. -2. -2. -2. -1. -2.  2.  2. -3. -2.  3. -3.  3. -2. -2. -2. -3.  3.
 -2. -2. -2.  3.  3. -1. -2. -1.  2. -2. -2. -1. -2. -2. -2. -3. -2.  3. -3. -2.  3. -2. -2. -2. -2. -1. -2. -1.  3.  3. -1. -2.  2. -3. -2. -2. -2.  3. -2. -2. -1.  2.  3.  3. -1.  2. -1. -2.  2.
  2. -3. -2.  3. -3. -2.  3.  2.  2. -1. -2. -2. -1.  2. -2.  3.  3. -1.  2. -3.  2.  2. -2.  3. -2. -2. -3.  3. -2. -1.  2. -2. -2. -3. -2.  3. -3. -2.  3.  2.  2.  2.  2. -3. -2.  3. -3.  3. -2.
 -2. -2. -1.  2.  2.  2.  3.  3. -1. -2. -1. -2. -2. -2. -1. -2. -3. -2. -2. -2.  3. -2. -2. -1.  2.  3.  3. -1.  2. -1. -2. -3.  2.  2. -2.  3. -2. -1. -2. -1.  2. -2.  3.  3. -1.  2. -3.  2.  2.
 -2.  3. -2. -2

In [4]:
import numpy as np

# Function to load a .mem file and convert it to Q3.4 fixed-point format
def load_q4_3_mem_file(filename):
    """
    Load a .mem file where each line is an 8-bit two's complement binary string
    in s[4][3] fixed-point format, and convert it to a numpy float array.
    """
    with open(filename, "r") as f:
        data = [CS.s4_3_to_decimal(line.strip()) for line in f]
    return np.array(data, dtype=np.float32)

# Function to decode Run-Length Encoding (RLE)
def run_length_decode(values, run_lengths):
    decoded = []
    for v, r in zip(values, run_lengths):
        decoded.extend([v] * r)
    return np.array(decoded, dtype=np.float32)  # Ensure floating-point format

# Load the `.mem` files with Q3.4 handling
values = load_q4_3_mem_file("bram/test_values.mem")       # Nonzero J values (Q3.4 fixed-point)
col_indices = np.loadtxt("bram/test_col_indices.mem", dtype=np.uint8, converters={0: lambda s: int(s, 2)})
row_ptr = np.loadtxt("bram/test_row_ptr.mem", dtype=np.uint16, converters={0: lambda s: int(s, 2)})
run_lengths = np.loadtxt("bram/test_run_lengths.mem", dtype=np.uint8, converters={0: lambda s: int(s, 2)})
h = load_q4_3_mem_file("bram/test_h.mem")            # Bias vector h (also in Q3.4 format)

# Decode the RLE `values[]`
decoded_values = run_length_decode(values, run_lengths)

# Verify length matches col_indices (they should be the same)
assert len(decoded_values) == len(col_indices), "Decoded values and col_indices mismatch!"

# Reconstruct sparse matrix
num_Pbits = len(h)  # The number of P-bits is determined by h
J_reconstructed = np.zeros((num_Pbits, num_Pbits), dtype=np.float32)  # Floating-point matrix

for i in range(num_Pbits):
    start_idx = row_ptr[i]
    end_idx = row_ptr[i + 1]
    
    for j in range(start_idx, end_idx):
        col = int(col_indices[j])  # Ensure index is integer
        J_reconstructed[i, col] = decoded_values[j]  # Assign Q3.4 converted value
np.set_printoptions(threshold=np.inf, linewidth=200)
# Print matrix for verification
print("Reconstructed J matrix (floating-point Q3.4):")
print(J_reconstructed)

# Matrix-vector multiplication test
m_test = np.random.choice([-1, 1], size=num_Pbits).astype(np.float32)  # Random ±1 states
weighted_sum = J_reconstructed @ m_test + h  # J * m + h

# Print results
print("\nMatrix-vector multiplication test:")
print("m_test:", m_test)
print("Weighted sum (J * m + h):", weighted_sum)


Reconstructed J matrix (floating-point Q3.4):
[[ 0.  0.  0.  0. -1. -1. -1. -1.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.
   0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.
   0.  0.  0.  0.]
 [-1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  2. 